In [1097]:
import numpy as np

## Logistic Regression Implementation

In [1098]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [1099]:
def calculate_gradient(theta, X, y):
    m = y.size      # This is the number of instances
    return (X.T @ (sigmoid(X @ theta) - y)) / m

In [1100]:
def gradient_descent(X, y, alpha=0.1, num_iterations=100, tol=1e-7):
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]

    theta = np.zeros(X_bias.shape[1])

    for i in range(num_iterations):
        gradient = calculate_gradient(theta, X_bias, y)
        theta -= alpha * gradient

        if np.linalg.norm(gradient) < tol:
            break

    return theta

In [1101]:
def predict_proba(X, theta):
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]

    return sigmoid(X_bias @ theta)


In [1102]:
def predict(X, theta, threshold=0.5):
    return (predict_proba(X, theta) >= threshold).astype(int)

In [1103]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [1104]:
iris_data = load_iris()
X, y = iris_data.data, iris_data.target

In [1105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [1106]:
# Training one classifier for each class using the One Vs Rest method (class 0 or non-class 0 and so on)
all_thetas = []
num_classes = len(np.unique(y))     # There are 3 unique classes in the Iris dataset

print(f"Training {num_classes} separate binary classifiers")

for i in range(num_classes):
    # Create a temporary target vector for this specific class
    y_train_specific = (y_train == i).astype(int)

    print(f"    Training classifier for class {i} ({iris_data.target_names[i]}) vs the rest")

    # Using the gradient descent function to train the binary classifier
    theta_hat = gradient_descent(X_train_scaled, y_train_specific, alpha=0.1, num_iterations=1000)

    # Store the trained parameters
    all_thetas.append(theta_hat)

print("Training complete")

Training 3 separate binary classifiers
    Training classifier for class 0 (setosa) vs the rest
    Training classifier for class 1 (versicolor) vs the rest
    Training classifier for class 2 (virginica) vs the rest
Training complete


In [1107]:
def predict_ovr(X, thetas):
    probabilities = np.array([predict_proba(X, theta) for theta in thetas]).T

    return np.argmax(probabilities, axis=1)

In [1108]:
# Predictions on the test set
y_pred = predict_ovr(X_test_scaled, all_thetas)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

# Classification report for a more detailed look
print("\nClassification report")
print(classification_report(y_test, y_pred, target_names=iris_data.target_names))


Model Accuracy: 0.9667

Classification report
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.89      0.94         9
   virginica       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30



## K Nearest Neighbors (KNN) Implementation

In [1109]:
from collections import Counter

In [1110]:
def calc_euclidean_dist(p, q):
    # This is a np.array of every value in p - every value in q
    #np.array(p) - np.array(q)

    return np.sqrt(np.sum((np.array(p) - np.array(q)) ** 2))

In [1111]:
class KNeighborsClassifier:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_trian, y_train):
        self.X_train = X_trian
        self.y_train = y_train

    def predict_single(self, x_test_point):
        '''Helper function to predict the label of one single data point'''

        # Calculate the distance to all training points
        distances = [calc_euclidean_dist(x_test_point, x_train_point) for x_train_point in self.X_train]

        # Get the indices of the k nearest neighbors
        k_neigbors_indices = np.argsort(distances)[:self.k]

        # Get the labels of the neighbors
        k_neigbors_labels = [self.y_train[i] for i in k_neigbors_indices]

        # Return the most common class label 
        most_common = Counter(k_neigbors_labels).most_common(1)
        return most_common[0][0]
    
    def predict(self, X_test):
        '''Makes predictions for a set of test data'''
        
        # Loop through each test point and predict its label
        preds = [self.predict_single(x_test_point) for x_test_point in X_test]
        return np.array(preds)
        

In [1112]:
# Loading and preparing the data
iris_data = load_iris()
X, y = iris_data.data, iris_data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
X_train_scld = scaler.fit_transform(X_train)
X_test_scld = scaler.transform(X_test)


In [1113]:
# Use the KNN classifier
knn_classifier = KNeighborsClassifier(k=5)

knn_classifier.fit(X_train_scld, y_train)

# Make predictions on the test set
y_predictions = knn_classifier.predict(X_test_scld)

In [1114]:
# Evaluate the model
acc = accuracy_score(y_test, y_predictions)

print(f"KNN model accuracy: {acc:.4f}\n")

print("Classification Report")
print(classification_report(y_test, y_predictions, target_names=iris_data.target_names))

KNN model accuracy: 0.9333

Classification Report
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       0.88      0.88      0.88         8
   virginica       0.89      0.89      0.89         9

    accuracy                           0.93        30
   macro avg       0.92      0.92      0.92        30
weighted avg       0.93      0.93      0.93        30

